In [ ]:
import pandas as pd
import faker

In [ ]:
# Crea un generatore di dati casuali
fake = faker.Faker()

# Leggi il file JSON
df_json = pd.read_json('../recipes/RAW_recipes_images_ingredients_cleaned1.json')

# Estrai gli ID unici
unique_ids = df_json['contributor_id'].unique()

# Lista per contenere i dati degli utenti
user_data = []
username_set = set()

# Genera dati casuali per ogni ID unico
for uid in unique_ids:
    first_name = fake.first_name()
    last_name = fake.last_name()
    # Inizia con un numero casuale per l'username
    num = fake.random_int(min=1, max=99)
    username = f"{first_name.lower()}_{last_name.lower()}_{num}"
    
    # Controlla l'unicità dell'username e aggiorna il numero se necessario
    while username in username_set:
        num += 1
        username = f"{first_name.lower()}_{last_name.lower()}_{num}"
    
    username_set.add(username)

    password = fake.password(length=10, special_chars=False)
    
    user_data.append({
        'Contributor_ID': uid,
        'Nome': first_name,
        'Cognome': last_name,
        'Username': username,
        'Password': password,
    })

# Crea un DataFrame Pandas
df = pd.DataFrame(user_data)

# Salva il DataFrame come file CSV
# df.to_json('utenti_generati.json', index=False, lines=False, orient='records', indent=4)


Si definisce la struttura degli utenti che andrà caricata su mongoDB. Bisogna aggiungere un array di post, in cui ogni post avrà la seguente struttura:
-   {
    "idPost" , ovvero recipeId
    "name", della ricetta
    "image", della ricetta
}

In [3]:
import pandas as pd
import json

# Carica i dati degli utenti dal file JSON
with open('utenti_generati.json', 'r') as file:
    utenti_data = json.load(file)

# Carica i dati dei post dal file JSON
with open('../recipes/RAW_recipes_images_ingredients_cleaned1.json', 'r') as file:
    posts_data = json.load(file)

# Crea un dataframe dagli utenti
df_utenti = pd.DataFrame(utenti_data)

# Crea un dataframe dai post e seleziona solo le colonne richieste
df_posts = pd.DataFrame(posts_data)[['contributor_id', 'id', 'name', 'Image']]
df_posts.rename(columns={'id': 'idPost'}, inplace=True)

# Aggiungi un array di post ad ogni utente
df_utenti['posts'] = df_utenti['Contributor_ID'].apply(
    lambda x: df_posts[df_posts['contributor_id'] == x].to_dict(orient='records'))

# Salva il dataframe in un file JSON
df_utenti.to_json('utenti_con_posts.json', orient='records', lines=False, indent=4)


In [4]:
with open('utenti_con_posts.json', 'r') as file:
    utenti = json.load(file)
    
for utente in utenti:
    for post in utente["posts"]:
        del post["contributor_id"]
        
with open('utenti_database.json', 'w') as file:
    json.dump(utenti, file, indent=4)